# Document Agent Prototype

This notebook contains examples for using the Document Agent Prototype.

It uses the new doc_agent_chat and doc_agent_upload services to upload documents to a project and then asks an agent questions about the documents.

In [1]:
import json
import subprocess
import os

## Upload a document

We can upload a document to the database by calling the doc_agent_upload service.

This service takes a URL to fetch text content, then it processes the text and uploads it to a vector database (Pinecone).

In [2]:
# The new services are designed to be called in a similar way to the existing AI chat services. This is just a function to illustrate the usage in this notebook.
def upload_document(doc_url, user_description, project_id):
    notebook_dir = os.path.dirname(os.path.abspath("__file__"))
    services_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
    
    input_data = {
        "doc_url": doc_url,
        "user_description": user_description,
        "project_id": project_id
    }
    
    input_path = os.path.join(services_dir, "tmp", "doc_agent_upload.json")
    output_path = os.path.join(services_dir, "tmp", "upload_output.json")
    
    with open(input_path, "w") as f:
        json.dump(input_data, f, indent=2)
    
    result = subprocess.run(
        ["python", "entry.py", "doc_agent_upload", "--input", "tmp/doc_agent_upload.json", "--output", "tmp/upload_output.json"],
        capture_output=True,
        text=True,
        cwd=services_dir
    )
    
    with open(output_path, "r") as f:
        output_data = json.load(f)
    
    print(f"✓ Uploaded: {user_description}")
    return output_data

The service takes a URL, user description and project ID. It returns a document ID for the uploaded document.

In [ ]:
# Example: upload a document
doc_url = "https://raw.githubusercontent.com/OpenFn/docs/refs/heads/main/docs/design/design-workflow.md"
user_description = "Design workflow"
project_id = "proj_1"

# Uncomment to upload:
# upload_result = upload_document(doc_url, user_description, project_id)

✓ Uploaded: Design workflow


## Chat with the document agent

We can chat to an LLM that has the ability to query the uploaded documents by calling the doc_agent_chat service.

In [3]:
def call_agent_chat(content, project_id, project_name, documents, history):
    # Build the input payload
    input_data = {
        "content": content,
        "context": {
            "project_id": project_id,
            "project_name": project_name,
            "documents": documents
        },
        "history": history
    }
    
    # Get the services directory (one level up from doc_agent_chat)
    notebook_dir = os.path.dirname(os.path.abspath("__file__"))
    services_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
    
    input_path = os.path.join(services_dir, "tmp", "doc_agent_chat.json")
    output_path = os.path.join(services_dir, "tmp", "output.json")
    
    # Write input file
    with open(input_path, "w") as f:
        json.dump(input_data, f, indent=2)
    
    # Call the service from the services directory
    result = subprocess.run(
        ["python", "entry.py", "doc_agent_chat", "--input", "tmp/doc_agent_chat.json", "--output", "tmp/output.json"],
        capture_output=True,
        text=True,
        cwd=services_dir
    )
    
    # Read the output
    with open(output_path, "r") as f:
        output_data = json.load(f)
    
    # Print only the response text
    if "response" in output_data:
        print("Response:")
        print("-" * 80)
        print(output_data["response"])
        print("-" * 80)
    
    return output_data

The document agent service takes a user message (content), project ID, project name, documents and conversation history as inputs.

Like the other Apollo chat services, this service is stateless and should be called every conversation turn. The front-end would need to have a system for storing project and document IDs at least, and the database a more sophisticated way to limit access between orgs/projects/docs.

In [4]:
# Query parameters
content = "How should I handle user data when working with the CLI?"
project_id = "proj_1"
project_name = "OpenFN CLI Project"

# All uploaded documents that the user wants to include in the conversation
documents = [
    {
        "uuid": "fd8370c4-24ad-4323-8ab2-4053569aedef",
        "title": "CLI Walkthrough",
        "description": "OpenFn CLI documentation"
    }
]
# Or use empty list if no documents:
# documents = []

history = []

# Call the service and save output
output = call_agent_chat(content, project_id, project_name, documents, history)

Response:
--------------------------------------------------------------------------------
Based on the OpenFN CLI documentation, here are the key guidelines for handling user data securely:

## Important Security Considerations

### Be Careful with Logging Sensitive Data



Note that `console.log(state)` will display the whole state, including `state.configuration` elements such as **username and password**. Remove this log whenever you're done debugging to avoid accidentally exposing sensitive information when the job is successfully deployed on production.





The OpenFn platform has built in protections to "scrub" state from the logs, but when you're using the CLI directly you're on your own!



### Store Credentials Securely



In the `workflow.json` file, specify a path to a git ignored configuration file that will contain necessary credentials that will be used to access the destination system.



For example:
```json
{
   "configuration": "tmp/openMRS-credentials.json"
}
```



## Inspect Citations and Retrieved Documents

Claude's Citations API provides precise references to source material. The response includes:

1. **Text blocks with citations**: Each claim in Claude's response can have citations attached
2. **Citation details**: Including document index, character/block locations, and the exact cited text
3. **Search results**: The document chunks that were retrieved from the vector database

The cell below shows how to parse and display all of this information.

In [9]:
output

{'response': 'Based on the OpenFN CLI documentation, here are the key guidelines for handling user data securely:\n\n## Important Security Considerations\n\n### Be Careful with Logging Sensitive Data\n\n\n\nNote that `console.log(state)` will display the whole state, including `state.configuration` elements such as **username and password**. Remove this log whenever you\'re done debugging to avoid accidentally exposing sensitive information when the job is successfully deployed on production.\n\n\n\n\n\nThe OpenFn platform has built in protections to "scrub" state from the logs, but when you\'re using the CLI directly you\'re on your own!\n\n\n\n### Store Credentials Securely\n\n\n\nIn the `workflow.json` file, specify a path to a git ignored configuration file that will contain necessary credentials that will be used to access the destination system.\n\n\n\nFor example:\n```json\n{\n   "configuration": "tmp/openMRS-credentials.json"\n}\n```\n\n### Separate Initial Data Files\n\n\n\nIn

In [ ]:
## Understanding Citations

# Show tool usage from history
print("=" * 80)
print("TOOL USAGE")
print("=" * 80)
print()

if "history" in output:
    for msg in output["history"]:
        if msg["role"] == "assistant" and "content" in msg:
            for item in msg["content"]:
                if isinstance(item, dict):
                    # Show thinking/text before tool use
                    if item.get("type") == "text":
                        text = item.get("text", "")
                        if text and "search" in text.lower():
                            print(f"💭 {text}")
                            print()
                    # Show tool calls
                    elif item.get("type") == "tool_use":
                        tool_name = item.get("name", "unknown")
                        tool_input = item.get("input", {})
                        print(f"🔧 Tool: {tool_name}")
                        print(f"   Input: {tool_input}")
                        print()

print()

# Build response with inline citation markers
print("=" * 80)
print("RESPONSE TEXT WITH INLINE CITATION MARKERS")
print("=" * 80)
print()

if "history" in output:
    last_message = output["history"][-1]
    if "content" in last_message and isinstance(last_message["content"], list):
        citation_counter = 1
        for block in last_message["content"]:
            if isinstance(block, dict) and block.get("type") == "text":
                text = block.get("text", "")
                citations = block.get("citations", [])
                
                # Print the text
                if citations:
                    # This text block has citations attached
                    print(f"{text} **[CITATION {citation_counter}]**")
                    citation_counter += len(citations)
                else:
                    # No citations for this block
                    print(text)

print("\n")

# Build a map of document_index to actual documents (for citation details)
document_map = {}
if "history" in output:
    doc_index = 0
    for msg in output["history"]:
        if msg["role"] == "user" and "content" in msg:
            for item in msg["content"]:
                if isinstance(item, dict) and item.get("type") == "tool_result":
                    tool_content = item.get("content", [])
                    if isinstance(tool_content, list):
                        for doc in tool_content:
                            if isinstance(doc, dict) and doc.get("type") == "document":
                                source_data = doc.get("source", {}).get("data", "")
                                context = doc.get("context", "")
                                document_map[doc_index] = {
                                    "index": doc_index,
                                    "text": source_data,
                                    "context": context
                                }
                                doc_index += 1

# Show detailed citations
if "citations" in output and output["citations"]:
    print("=" * 80)
    print(f"CITATION DETAILS ({len(output['citations'])} total)")
    print("=" * 80)
    print()
    
    for i, citation in enumerate(output["citations"], 1):
        doc_idx = citation.get('document_index')
        print(f"[CITATION {i}]")
        print(f"  References Document Index: {doc_idx}")
        
        if doc_idx in document_map:
            print(f"  Document Context: {document_map[doc_idx]['context']}")
        
        print(f"  Type: {citation.get('type', 'unknown')}")
        
        # Show location information
        if citation.get('type') == 'char_location':
            start = citation.get('start_char_index', '?')
            end = citation.get('end_char_index', '?')
            print(f"  Character Range: {start}-{end}")
        elif citation.get('type') == 'content_block_location':
            start = citation.get('start_block_index', '?')
            end = citation.get('end_block_index', '?')
            print(f"  Block Range: {start}-{end}")
        
        # Show full cited text
        cited_text = citation.get('cited_text', '')
        print(f"  Cited Text:")
        for line in cited_text.split('\n'):
            print(f"    {line}")
        print()

# Show search results summary
if "meta" in output and "search_results" in output["meta"]:
    search_results = output["meta"]["search_results"]
    print("=" * 80)
    print(f"SEARCH RESULTS SUMMARY ({len(search_results)} chunks retrieved from vector DB)")
    print("=" * 80)
    print()
    
    for i, result in enumerate(search_results, 1):
        metadata = result.get('metadata', {})
        print(f"Result {i}:")
        print(f"  Score: {result.get('score', '?')}")
        print(f"  Document: {metadata.get('doc_title', 'Unknown')}")
        print(f"  Description: {metadata.get('user_description', 'N/A')}")
        print(f"  Text preview: {result.get('text', '')[:80]}...")
        print()